# Automatically Generate a PDF and send it by Email

Overview

A company that sells second hand cars, management wants to get a summary of the amounts of vehicles that have been sold at the end of every month. The company already has a web service which serves sales data at the end of every month but management wants an email to be sent out with an attached PDF so that data is more easily readable.

What will be done

Write a script that summarizes and processes sales data into different categories
Generate a PDF using Python
Automatically send a PDF by email

In [ ]:
#!/usr/bin/env python3
import emails
import json
import locale
import sys
import reports

def load_data(filename):
  """Loads the contents of filename as a JSON file."""
  with open(filename) as json_file:
    data = json.load(json_file)
  return data


def format_car(car):
  """Given a car dictionary, returns a nicely formatted name."""
  return "{} {} ({})".format(
      car["car_make"], car["car_model"], car["car_year"])


def process_data(data):
  """Analyzes the data, looking for maximums.

  Returns a list of lines that summarize the information.
  """
  max_revenue = {"revenue": 0}
  max_sales = {"sales": 0}
  year_count = {}
  for item in data:
    # Calculate the revenue generated by this model (price * total_sales)
    # We need to convert the price from "$1234.56" to 1234.56
    item_price = locale.atof(item["price"].strip("$"))
    item_revenue = item["total_sales"] * item_price
    if item_revenue > max_revenue["revenue"]:
      item["revenue"] = item_revenue
      max_revenue = item
    if item["total_sales"] > max_sales["sales"]:
      max_sales = {"sales": item["total_sales"], "car": item["car"]}
    year = item["car"]["car_year"]
    if year not in year_count:
      year_count[year] = 0
    year_count[year] += item["total_sales"]

  popular_year = max(year_count, key=year_count.get)
  summary = [
    "The {} generated the most revenue: ${}".format(
      format_car(max_revenue["car"]), max_revenue["revenue"]),
    "The {} had the most sales: {}".format(
      format_car(max_sales["car"]), max_sales["sales"]),
    "The most popular year was {} with {} sales.".format(
      popular_year, year_count[popular_year])
  ]

  return summary


def cars_dict_to_table(car_data):
  """Turns the data in car_data into a list of lists."""
  table_data = [["ID", "Car", "Price", "Total Sales"]]
  for item in car_data:
    table_data.append([item["id"], format_car(item["car"]), item["price"], item["total_sales"]])
  return table_data

def generate_report(filename, title, additional_info, table_data):
    reports.generate(filename, title, "<br/>".join(additional_info), table_data)

def send_email():
    sender = "automation@example.com"
    recipient = "student-01-68befa7368a3@example.com"
    subject = "Sales summary for last month"
    body = "The same summary from the PDF, but using \n between the lines"
    attachment_path = "/tmp/cars.pdf"
    message = emails.generate(sender, recipient, subject, body, attachment_path)
    emails.send(message)

def main(argv):
  """Process the JSON data and generate a full report out of it."""
  data = load_data("car_sales.json")
  summary = process_data(data)
  table_data = cars_dict_to_table(data)
  generate_report('/tmp/cars.pdf', 'Sales summary for last month', summary, table_data)
  send_email()

if __name__ == "__main__":
  main(sys.argv)